In [27]:
import openai
import pandas as pd
import os
import json

In [28]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [29]:
prompt = '''Genera una pequeña descripcion de producto y un posible nonbre de compañia de acuerdo al input, la salida debe ser en un formato JSON
ejemplo:
###
Input: auto
output: 'Descripcion':'Auto electrico de color rojo, amigable con el medio ambiente'
 'compañia': 'EcoCar'
###
 
input:{}
output en formato JSON:
'''

In [30]:
def return_productos(product):
    completion = openai.ChatCompletion.create(
        model="gpt-4o",  # O el modelo que tengas disponible
        messages=[
            {"role": "user", "content": prompt.format(product)},
        ],
        temperature=0
    )
    
    raw_response = completion.choices[0].message["content"].strip()
    
    # Intentar extraer solo el JSON
    try:
        return json.loads(raw_response)
    except json.JSONDecodeError:
        import re
        json_match = re.search(r"\{.*\}", raw_response, re.DOTALL)
        if json_match:
            return json.loads(json_match.group(0))
        else:
            raise ValueError(f"No se pudo extraer JSON de la respuesta: {raw_response}")

In [31]:
return_productos('silla')

{'Descripcion': 'Silla ergonómica de diseño moderno, ideal para oficinas y espacios de trabajo, fabricada con materiales reciclables.',
 'compañia': 'EcoSeating'}

In [32]:
## Crea una lista de 10 productos de clases distintas
productos = [
    'silla',
    'mesa',
    'laptop',
    'teléfono móvil',
    'reloj inteligente',
    'bicicleta',
    'auriculares',
    'televisor',
    'cámara fotográfica',
    'impresora'
]

In [33]:
list_desc = []
list_prod = []
for _ in productos:
    try:
        list_desc.append(return_productos(_))
        list_prod.append(_)
    except:
        None

In [35]:
df = pd.DataFrame({'producto': list_prod, 'descripciones': list_desc})

In [36]:
df

,producto,descripciones
0,silla,{'Descripcion': 'Silla ergonómica de diseño mo...
1,mesa,"{'Descripcion': 'Mesa de madera de roble, idea..."
2,laptop,{'Descripcion': 'Laptop ultraligera con pantal...
3,teléfono móvil,{'Descripcion': 'Teléfono móvil de última gene...
4,reloj inteligente,{'Descripcion': 'Reloj inteligente con monitor...
5,bicicleta,{'Descripcion': 'Bicicleta de montaña con marc...
6,auriculares,{'Descripcion': 'Auriculares inalámbricos con ...
7,televisor,{'Descripcion': 'Televisor 4K Ultra HD con pan...
8,cámara fotográfica,{'Descripcion': 'Cámara fotográfica digital de...
9,impresora,{'Descripcion': 'Impresora multifuncional de a...


In [37]:
def split_json(df):
    df['Descripcion'] = df['descripciones'].apply(lambda x: x.get('Descripcion', ''))
    df['compañia'] = df['descripciones'].apply(lambda x: x.get('compañia', ''))
    df.drop(columns=['descripciones'], inplace=True)
    return df

df = split_json(df)

In [38]:
df

,producto,Descripcion,compañia
0,silla,"Silla ergonómica de diseño moderno, ideal para...",ConfortEco
1,mesa,"Mesa de madera de roble, ideal para comedores ...",MueblesArmonía
2,laptop,Laptop ultraligera con pantalla táctil de alta...,TechWave
3,teléfono móvil,Teléfono móvil de última generación con pantal...,TechWave
4,reloj inteligente,"Reloj inteligente con monitor de salud, notifi...",TechTime
5,bicicleta,Bicicleta de montaña con marco de aluminio lig...,MountainRide
6,auriculares,Auriculares inalámbricos con cancelación de ru...,SoundWave
7,televisor,"Televisor 4K Ultra HD con pantalla OLED, ofrec...",VisionTech
8,cámara fotográfica,"Cámara fotográfica digital de alta resolución,...",FotoVision
9,impresora,"Impresora multifuncional de alta velocidad, id...",PrintEco Solutions
